## Разведочный анализ данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn

In [2]:
ds_control = pd.read_csv('control_data.csv')

Рассмотрим имеющиеся в нашем распоряжении данные:

In [3]:
ds_control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


Составим интересующие нас метрики по имеющимся данным:

In [4]:
ds_control['CTP'] = round(ds_control['Clicks']/ds_control['Pageviews'], 3)
ds_control['GC'] = round(ds_control['Enrollments']/ds_control['Clicks'], 3)
ds_control['Retention'] = round(ds_control['Payments']/ds_control['Enrollments'], 3)
ds_control['NC'] = round(ds_control['Payments']/ds_control['Clicks'], 3)
ds_control['PP'] = round(ds_control['Payments']/ds_control['Pageviews'], 3)
ds_control.head()

,Date,Pageviews,Clicks,Enrollments,Payments,CTP,GC,Retention,NC,PP
0,"Sat, Oct 11",7723,687,134.0,70.0,0.089,0.195,0.522,0.102,0.009
1,"Sun, Oct 12",9102,779,147.0,70.0,0.086,0.189,0.476,0.090,0.008
2,"Mon, Oct 13",10511,909,167.0,95.0,0.086,0.184,0.569,0.105,0.009
3,"Tue, Oct 14",9871,836,156.0,105.0,0.085,0.187,0.673,0.126,0.011
4,"Wed, Oct 15",10014,837,163.0,64.0,0.084,0.195,0.393,0.076,0.006


Проведем разведочный анализ данных. 
Для этого воспользуемся библиотекой pandas_profiling, 
которая позволяет получить все необходимые для первичного анализа сведения в формате HTML-отчета:
1. тип данных в каждой колонке;
2. пропущенные и уникальные значения (количество и процент);
3. описательные статистики: квартили, медиану, 
    межквартильный размах, среднее, моду, абсолютное и 
    относительное стандартное отклонение, медианное абсолютное отклонение, 
    коэффициенты асимметрии и эксцесса;
4. графики распределений в виде гистограммы частот;
5. корреляции между значениями (Пирсона, Спирмена и Кендалла);
6. матрица пропущенных значений.

In [5]:
import pandas_profiling
pandas_profiling.ProfileReport(ds_control)

ModuleNotFoundError: No module named 'pandas_profiling'

### Выводы


<p>
Благодаря отчету, полученному с помощью pandas_profiling можно сделат несколько полезных для нас выводов:
    <br>
    <br>
    1. Глядя на описательные статистики и гистограммы видно, что наши данные (Date, Pageviews, Clicks, Enrollments,Payments) не содержат выбросов. Поэтому во время проведения AB-теста их ждать не стоит, что облегчает нам задачу. 
    <br>
    <br>
    2. Гистограммы частот говорят о том, что половина распределений наших данных принадлежат к классу нормальных (Pageviews, Enrollments), а распределение другой половины напоминает раномерное (Clicks, Payments).  
    <br>
    <br>
    3. Глядя на построенную матрицу корреляций, а также на специальные сообщения (allerts) замечаем, что многие из наших 
    данных сильно корреляируют друг с другом. Например число кликов по кнопке с регистрацией на пробный период (Clicks) зависит от числа просмотров страницы курса (Pageviews), число платящих пользователей (Payments) зависит от числа регистраций на курс (Enrollements). Стоит отметить, что хотя наличие корреляции не влияет на алгоритм проведения AB-теста, но полученные зависимости расширяют наши знания о предметной области.
    <br>
    <br>
    4.Глядя на матрицу пропущенных значений, видим, что они у нас имеются, причем носят систематический характер и их 
    довольно много (37.8% значений отсутствует у Enrollements и Payments, поэтому столько же пропусков имеют и наши метрики (GC, Retention, NC, PTP), которые получаются из них простыми арифметическими преобразованиями. Это необходимо учитывать при проведении AB-теста.
</p>